<a href="https://colab.research.google.com/github/kimdonggyu2008/combined/blob/main/Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import re
from transformers import BartForConditionalGeneration, BartTokenizer
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import pandas as pd

# BART 모델 및 토크나이저 초기화
summarizer_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
summarizer_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 모델 초기화
def load_translation_models():
    """
    NLLB 모델과 토크나이저를 초기화합니다.
    """
    languages = ['fra', 'deu', 'rus', 'jpn']  # 프랑스어, 독일어, 러시아어, 일본어
    models = {}
    base_model = "facebook/nllb-200-3.3B"  # NLLB 대형 모델 (3.3B 버전)

    # NLLB 모델과 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(base_model)
    model = AutoModelForSeq2SeqLM.from_pretrained(base_model)

    # 각 언어 쌍 초기화
    for lang in languages:
        models[f"eng_{lang}"] = {"model": model, "tokenizer": tokenizer, "target_lang": lang}
        models[f"{lang}_kor"] = {"model": model, "tokenizer": tokenizer, "target_lang": "kor"}
    return models
translation_models = load_translation_models()


In [ ]:

def detect_site(url):
    """
    URL을 분석하여 사이트를 감지합니다.
    """
    parsed_url = urlparse(url)
    domain = parsed_url.netloc

    if "bbc" in domain:
        return "bbc"
    elif "cnn" in domain:
        return "cnn"
    else:
        return None

def get_filtered_main_content(url):
    """
    URL에서 원문(article)을 크롤링합니다.
    """
    content = ""

    # HTML 소스 가져오기
    response = requests.get(url)
    if response.status_code != 200:
        return f"Failed to fetch the webpage! URL: {url}"

    # 인코딩 설정
    response.encoding = response.apparent_encoding

    soup = BeautifulSoup(response.content, 'html.parser')

    # 사이트 감지
    site = detect_site(url)
    if site == "bbc":
        text_blocks = soup.select('div[data-component="text-block"]')
        for block in text_blocks:
            content += block.get_text(strip=True) + "\n\n"
    elif site == "cnn":
        paragraphs = soup.select("p.paragraph.inline-placeholder.vossi-paragraph")
        for paragraph in paragraphs:
            content += paragraph.get_text(strip=True) + "\n\n"
    else:
        return None  # 지원되지 않는 사이트

    # 텍스트 정리
    content = (
        content.replace("\s\s+", " ")
        .replace(". ", ".\n\n")
        .strip()
    )

    return content or None

def split_text_with_last_sentence_overlap(text, target_chunk_length=1024):
    """
    긴 텍스트를 지정된 길이로 문장 단위로 분할하며, 마지막 문장을 중복 포함시킵니다.
    """
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk.split()) + len(sentence.split()) <= target_chunk_length:
            current_chunk += sentence + " "
        else:
            # 현재 청크를 저장하고 다음 청크로 이동, 마지막 문장 포함
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def summarize_long_text(article_text, target_chunk_length=1024):
    """
    긴 텍스트를 분할하여 각 청크를 요약하고 결합합니다.

    Args:
        article_text (str): 원문 텍스트.
        target_chunk_length (int): 각 청크의 최대 토큰 수.

    Returns:
        str: 전체 텍스트 요약 결과.
    """
    # 텍스트 분할
    chunks = split_text_with_last_sentence_overlap(article_text, target_chunk_length)
    summaries = []

    for chunk in chunks:
        try:
            # 텍스트 토크나이징 및 모델 입력 생성
            inputs = summarizer_tokenizer(chunk, max_length=target_chunk_length, return_tensors="pt", truncation=True)

            # 요약 생성
            summary_ids = summarizer_model.generate(
                inputs["input_ids"],
                max_length=100,   # 출력 요약 최대 길이
                min_length=50,    # 출력 요약 최소 길이
                length_penalty=2.0,
                num_beams=2,
                early_stopping=True
            )
            # 요약 디코딩
            summary = summarizer_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            summaries.append(summary)
        except Exception as e:
            print(f"Error processing chunk: {chunk[:100]}... | Error: {e}")
            summaries.append("Summarization failed for this chunk.")

    # 요약된 청크 결합
    return " ".join(summaries)

def process_urls_from_file(file_path, output_path):
    """
    URL이 담긴 파일을 처리하여 요약 데이터를 생성하고 저장합니다.
    """
    results = []

    # TXT 파일에서 URL 읽기
    with open(file_path, 'r') as file:
        urls = file.read().split(',')

    for url in urls:
        url = url.strip()  # 공백 제거
        if not url:
            continue

        print(f"Processing URL: {url}")
        article = get_filtered_main_content(url)
        if not article:
            print(f"Skipping URL (no article found): {url}")
            continue

        # 요약 생성
        highlight = summarize_long_text(article)

        # 크롤링 결과 저장
        results.append({
            "url": url,
            "article": article,
            "highlight": highlight
        })

    # 결과를 DataFrame으로 변환
    df = pd.DataFrame(results)

    # 데이터 저장 (CSV 형식으로 저장)
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Results saved to {output_path}")

# 실행 예제
if __name__ == "__main__":
    input_file = "/content/drive/MyDrive/summarizer/data/test_summary.txt"  # URL이 담긴 TXT 파일 경로
    output_file = "/content/drive/MyDrive/summarizer/data/crawled_summary.csv"  # 결과 CSV 파일 경로

    process_urls_from_file(input_file, output_file)


Processing URL: https://www.bbc.com/news/articles/cd6vpy8e6jvo
Processing URL: https://www.bbc.com/news/articles/cly20zz51j7o
Processing URL: https://www.bbc.com/future/article/20241115-how-robotaxis-are-trying-to-win-passengers-trust
Processing URL: https://www.bbc.com/news/articles/c9wrqg4vd2qo
Processing URL: https://www.bbc.com/news/articles/c5yxv41q235o
Processing URL: https://www.bbc.com/news/articles/c30p16gn3pvo
Processing URL: https://www.bbc.com/news/articles/cdenplz5j89o
Processing URL: https://www.bbc.com/future/article/20230306-just-how-loud-is-a-rocket-launch
Processing URL: https://www.bbc.com/future/article/20241111-stressed-writing-down-a-to-do-list-might-help
Processing URL: https://www.bbc.com/future/article/20201028-the-benefits-of-coffee-is-coffee-good-for-health


In [ ]:


def translate_text(text, source_lang, target_lang, model_key):
    """
    텍스트를 지정된 언어로 번역합니다.
    """
    if model_key not in translation_models:
        raise ValueError(f"No translation model for {source_lang} -> {target_lang}")

    model_info = translation_models[model_key]
    tokenizer = model_info["tokenizer"]
    model = model_info["model"]

    # 번역 수행
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs["forced_bos_token_id"] = tokenizer.lang_code_to_id[model_info["target_lang"]]  # 대상 언어 설정
    outputs = model.generate(inputs["input_ids"], max_length=512, num_beams=4, early_stopping=True)
    translated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    return translated_text

def create_translation_dataset(summary_df, output_file):
    """
    요약 데이터를 다양한 언어로 번역 후 다시 한국어로 역번역한 데이터셋을 생성합니다.

    Args:
        summary_df (pd.DataFrame): 요약 데이터가 포함된 데이터프레임. (columns: "url", "article", "highlight")
        output_file (str): 결과 CSV 파일 경로.
    """
    results = []

    for _, row in summary_df.iterrows():
        original_summary = row["highlight"]  # 영어 원문 요약
        translations = {"en": original_summary}  # 원문은 'en' 열에 추가

        for lang in ['fra', 'deu', 'rus', 'jpn']:  # 프랑스어, 독일어, 러시아어, 일본어
            try:
                # 영어 -> 중간 언어
                intermediate_translation = translate_text(
                    original_summary, "eng", lang, f"eng_{lang}"
                )
                # 중간 언어 -> 한국어
                back_translation = translate_text(
                    intermediate_translation, lang, "kor", f"{lang}_kor"
                )
                translations["ko"] = back_translation  # 한국어 역번역 결과 추가
            except Exception as e:
                print(f"Translation failed for {lang}: {e}")
                translations["ko"] = "Translation failed"

            # 'ko'와 'en' 열만 유지
            results.append({
                "ko": translations["ko"],
                "en": translations["en"]
            })

    # 결과를 데이터프레임으로 변환 후 저장
    translation_df = pd.DataFrame(results)
    translation_df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"Translated dataset saved to {output_file}")

# 실행 예제
if __name__ == "__main__":
    # 요약 데이터 읽기
    summary_data_file = "/content/drive/MyDrive/summarizer/data/crawled_summary.csv"
    summary_df = pd.read_csv(summary_data_file)

    # 번역 데이터셋 생성
    output_translation_file = "/content/drive/MyDrive/summarizer/data/translated_summary.csv"
    create_translation_dataset(summary_df, output_translation_file)

